In [ ]:
# Assignment: Longest Common Subsequence (HPC Project)
#**Student**: Parrella Marco, Matricola: 0622702536, Email: m.parrella21@studenti.unisa.it
#**Lecturer**: Moscato Francesco, fmoscato@unisa.it

#**License**: GPLv3 (see LICENSE file)
#**Requirements**: Implement Parallel LCS (OpenMP, MPI, CUDA)
#**Purpose**: Notebook per la compilazione e l'esecuzione dei benchmark della versione CUDA su ambiente con GPU (Google Colab).

import os
import subprocess
import sys

# --- ISTRUZIONI PRELIMINARI ---
# 1. Caricare il file 'src/lcs_cuda.cu' nella cartella principale di Colab.
# 2. Caricare il file 'data/generate_input.py' nella cartella principale di Colab.

# Verifica presenza file necessari
required_files = ["lcs_cuda.cu", "generate_input.py"]
missing = [f for f in required_files if not os.path.exists(f)]
if missing:
    print(f"ERRORE: Mancano i seguenti file nell'ambiente Colab: {missing}")
    print("Per favore caricali trascinandoli nella barra laterale 'Files'.")
    sys.exit(1)
else:
    print("File sorgenti trovati. Procedo con la compilazione.")

# --- PASSO 1: COMPILAZIONE ---
print("Compilazione in corso con NVCC...")
!nvcc -O3 lcs_cuda.cu -o lcs_cuda
if os.path.exists("lcs_cuda"):
    print("Compilazione completata con successo.\n")
else:
    print("ERRORE durante la compilazione.")
    sys.exit(1)


# --- PASSO 2: BENCHMARK ---
SIZES = [10240, 51200]  # 10KB, 50KB (Aggiungi altre size se necessario)
BLOCK_SIZES = [128, 256, 512]
REPS = 3

print("impl,size,rep,param,time_s")

for s in SIZES:
    file_a = f"bench_{s}_A.bin"
    file_b = f"bench_{s}_B.bin"

    # Generazione dati se non esistono
    if not os.path.exists(file_a):
        !python3 generate_input.py --size-per-file $s --prefix "bench_{s}" --seed 42 --alphabet ascii

    for r in range(1, REPS + 1):
        for b in BLOCK_SIZES:
            try:
                cmd = f"./lcs_cuda {file_a} {file_b} {b}"
                # Eseguiamo il comando e catturiamo l'output
                output = subprocess.check_output(cmd, shell=True).decode("utf-8")

                for line in output.splitlines():
                    if "ELAPSED_TIME:" in line:
                        time_s = line.split(":")[1].strip()
                        print(f"cuda,{s},{r},{b},{time_s}")
            except subprocess.CalledProcessError as e:
                print(f"cuda,{s},{r},{b},ERROR")

Codice lcs_cuda.cu aggiornato con supporto per matrici grandi (size_t).

Compilazione in corso...
nvcc warning : Support for offline compilation for architectures prior to '<compute/sm/lto>_75' will be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
Compilazione completata.

impl,size,rep,param,time_s
cuda,10240,1,128,0.109924
cuda,10240,1,256,0.064252
cuda,10240,1,512,0.076628
cuda,10240,2,128,0.060675
cuda,10240,2,256,0.064462
cuda,10240,2,512,0.076243
cuda,10240,3,128,0.065312
cuda,10240,3,256,0.064488
cuda,10240,3,512,0.076185
cuda,51200,1,128,2.992495
cuda,51200,1,256,3.016323
cuda,51200,1,512,3.072156
cuda,51200,2,128,2.995034
cuda,51200,2,256,3.016395
cuda,51200,2,512,3.078468
cuda,51200,3,128,3.002448
cuda,51200,3,256,3.023699
cuda,51200,3,512,3.081815
